# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
pip install pymystem3==0.1.10

     |████████████████████████████████| 61 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 3.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [4]:
solvency_data = pd.read_csv('./borrowers_reliability.csv')

solvency_data.info()
solvency_data.sample(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15764,0,-1743.052642,48,среднее,1,женат / замужем,0,M,сотрудник,1,171879.095992,профильное образование
12836,1,-2173.355642,34,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,111129.991656,свой автомобиль
15970,0,-6160.820391,45,среднее,1,женат / замужем,0,F,сотрудник,1,121925.930427,получение образования
2716,0,-1516.375426,23,среднее,1,гражданский брак,1,M,сотрудник,0,95025.635979,свадьба
395,0,-437.448857,29,высшее,0,женат / замужем,0,F,сотрудник,0,189608.445764,заняться высшим образованием
6482,0,-2088.072298,40,среднее,1,женат / замужем,0,F,сотрудник,0,78828.773632,ремонт жилью
1778,1,-4815.649328,50,высшее,0,вдовец / вдова,2,F,госслужащий,0,170622.718045,покупка коммерческой недвижимости
18184,1,-1963.483535,27,среднее,1,женат / замужем,0,F,компаньон,0,145521.977288,получение дополнительного образования
12082,0,-2466.408852,34,среднее,1,женат / замужем,0,M,компаньон,0,472899.096902,на покупку своего автомобиля
9439,1,NaN,47,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции с коммерческой недвижимостью


**Вывод**

В таблице по исследованию платёжеспособности заёмщиков присутствует 21525 записей, имеющих по 12 опреледяющих их признаков со следующими типами данных: 2 float, 5 integer, 5 object. Столбцы имеют унифицированные имена, однако dob_years и total_income нуждаются на мой взгляд в переименовании в более разьясняющие названия как full_years и monthly_income. Если обратить внимание на столбцы days_employed и total_income, то можно заметить, что оба имеют nullable значения, в отличии от других столбцов, при этом у обоих по 19351 non-null значений плюс если у одного человнка стоит NaN в days_employed, то тоже значение и в total_income, из чего можно сделать предположение, что данные о работе были вероятнее всего по каким-то причинам, толи скрыты, толи утеряным при выборке. 

При наличии большого колличества данных (в данном случае это лишь около 2MB), можно было бы сохранить память переведя children, education_id, family_status_id, debt из int64 в unsigned int8, так как диапозон значений мал и не может быть отрицательным. Для income_type можно было бы также создать ассоциативный столбик с id, как это сделано в education_id, family_status_id, а сами полные обозначения этих столбцов перенести в отдельные таблицы. 

Столбец days_employed вообще по своей сути немного аномальный, так как дни не могут быть отрицательными значениями, да и для некоторых записей колличество дней во много раз больше нежели общее количество дней прожитых человеком, что собственно говоря невозможно. Также данный столбец имеет тип float64, что может быть вызвано неполными рабочими днями, но все же нам нужно будет перевести его в int64, так как с днями удобнее работать в целочисленной форме. Аномалии в других столбцах замечены не были, единственное total_income можно также перевести в int64, так как будет удобнее работать с целочисленной денежной еденицей, без ее милионных.

### Переименование столбцов

In [4]:
# dob_years и total_income переименуем в более разьясняющие названия как full_years и monthly_income
solvency_data = solvency_data.rename(columns={"dob_years": "full_years", "total_income": "monthly_income"}) 
solvency_data.columns

Index(['children', 'days_employed', 'full_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'monthly_income', 'purpose'],
      dtype='object')

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# Проверка колличества пропусков в стаже по отношению к пропускам в стаже и месячной зарплате
nullable_days_epmloyed = solvency_data[solvency_data['days_employed'].isna()]
nullable_days_emloyed_and_monthly_income = solvency_data[solvency_data['days_employed'].isna() & solvency_data['monthly_income'].isna()]

nullable_days_emloyed_to_all = len(nullable_days_epmloyed) / len(solvency_data)
nullable_days_emloyed_and_monthly_income_to_all = len(nullable_days_emloyed_and_monthly_income) / len(solvency_data)

print("Процент людей с пропусками в стаже: {:.2%}".format(nullable_days_emloyed_to_all))
print("Процент людей с пропусками в стаже и зарплате: {:.2%}".format(nullable_days_emloyed_and_monthly_income_to_all))

Процент людей с пропусками в стаже: 10.10%
Процент людей с пропусками в стаже и зарплате: 10.10%


In [6]:
# Применим модуль полю days_employed, так как дни не могут иметь отрицательное значение
solvency_data['days_employed'] = solvency_data['days_employed'].abs()

# Заполним аномальные значения с помощью NaN
YEAR_IN_DAYS = 365
days_employed_to_age_compare_mask = solvency_data['days_employed'] > solvency_data['full_years'] * YEAR_IN_DAYS
solvency_data.loc[days_employed_to_age_compare_mask, 'days_employed'] = np.nan

# Заменим NaN в days_employed и monthly_income помощью медианы, 
# чтобы не учитывать эестремумы и больше ориентироваться на средние значения рабочнго стажа и месячной зарплаты
median_monthly_income = solvency_data['monthly_income'].median()
median_days_employed = solvency_data['days_employed'].median()
print("Медианная месячная зарплата: {}".format(median_monthly_income))
print("Медианный стаж работы в днях: {}".format(median_days_employed))

solvency_data['monthly_income'] = solvency_data['monthly_income'].fillna(median_monthly_income)
solvency_data['days_employed'] = solvency_data['days_employed'].fillna(median_days_employed)

solvency_data.sample(50)

Медианная месячная зарплата: 145017.93753253992
Медианный стаж работы в днях: 1630.6913715168535


,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,monthly_income,purpose
16261,0,1630.691372,59,среднее,1,женат / замужем,0,M,пенсионер,0,128775.662658,строительство собственной недвижимости
19633,0,223.832560,22,среднее,1,гражданский брак,1,F,сотрудник,0,205789.055638,строительство недвижимости
6651,0,828.374131,37,среднее,1,в разводе,3,M,сотрудник,1,162000.325735,сделка с автомобилем
2679,0,568.444798,40,среднее,1,гражданский брак,1,M,сотрудник,0,179082.674877,автомобили
21359,0,1630.691372,63,среднее,1,женат / замужем,0,M,пенсионер,0,89275.261302,покупка жилья для сдачи
6150,0,941.587816,58,среднее,1,гражданский брак,1,F,сотрудник,0,79379.269525,на проведение свадьбы
7441,0,2712.999962,42,среднее,1,в разводе,3,F,сотрудник,0,136478.643244,получение дополнительного образования
12688,0,508.575444,44,среднее,1,женат / замужем,0,M,сотрудник,1,241658.940982,покупка жилья для семьи
884,0,2308.271667,35,высшее,0,женат / замужем,0,M,госслужащий,0,673040.439784,на покупку своего автомобиля
4309,2,1630.691372,31,неоконченное высшее,2,женат / замужем,0,F,госслужащий,0,145017.937533,приобретение автомобиля


**Вывод**

Исходя из общей информации о таблице мы увидели, что NaN присутствуют лишь в двух столбцах days_employed и monthly_income. При анализе пропусков обратили внимание, что количество людей с пропусками в стаже и с пропусками в стаже + зарплате равно 10,1%, что привело нас к выводу о наличии этих пропусков у одних и тех же людей. Как говорилось ранее, причиной таких значений могут быть: утеря данных при извлечении либо их вероятное скрытие. Пропуски были заменены медианным значением, чтобы не учитывать экстремумы. 

### Замена типа данных

In [7]:
# Заменим для столбцов days_employed и monthly_income значения с вещественных (float64) на целочисленные (int64)
print('Тип данных столбца days_employed до преобразования: {}'.format(solvency_data['days_employed'].dtype))
print('Тип данных столбца monthly_income до преобразования: {}'.format(solvency_data['monthly_income'].dtype))

solvency_data['days_employed'] = solvency_data['days_employed'].astype('int64')
solvency_data['monthly_income'] = solvency_data['monthly_income'].astype('int64')

print('Тип данных столбца days_employed после преобразования: {}'.format(solvency_data['days_employed'].dtype))
print('Тип данных столбца monthly_income после преобразования: {}'.format(solvency_data['monthly_income'].dtype))
solvency_data.sample(50)

Тип данных столбца days_employed до преобразования: float64
Тип данных столбца monthly_income до преобразования: float64
Тип данных столбца days_employed после преобразования: int64
Тип данных столбца monthly_income после преобразования: int64


,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,monthly_income,purpose
6538,0,602,44,среднее,1,Не женат / не замужем,4,F,компаньон,0,96541,покупка коммерческой недвижимости
16156,1,891,35,среднее,1,женат / замужем,0,M,сотрудник,0,102098,покупка своего жилья
7506,1,1630,52,среднее,1,женат / замужем,0,F,пенсионер,0,229668,покупка жилья для сдачи
11051,2,327,36,среднее,1,женат / замужем,0,M,компаньон,0,172549,дополнительное образование
904,1,1853,47,среднее,1,гражданский брак,1,M,сотрудник,0,85242,свадьба
1501,0,3402,27,среднее,1,женат / замужем,0,F,сотрудник,0,161281,заняться высшим образованием
16760,0,676,55,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,97096,сыграть свадьбу
13710,0,1630,26,среднее,1,женат / замужем,0,F,компаньон,0,145017,строительство жилой недвижимости
8724,0,648,48,среднее,1,женат / замужем,0,F,сотрудник,0,159518,покупка жилья для семьи
15411,2,546,36,среднее,1,женат / замужем,0,F,сотрудник,0,217312,на покупку подержанного автомобиля


**Вывод**

Для столбцов days_employed и monthly_income значения были заменены с вещественных на целочисленные, так как с днями и зарплатой удобнее работать без учета нецелой части и в принципе это улучшает восприятее данных, чтобы не отображать части дня и копейки, да и на точность данных в данном исслеловании это не повлияет.

### Обработка дубликатов

In [8]:
# Посмотрим уникадиные значения в интересующих нас типах данных 
# для поиска возможных дубликатов категорий и их перевода в единую форму регистра
# а также наличия каких-то оставшихся аномалий

def get_unique_values_and_their_amount_for_column(column_name):
    unique_values = solvency_data[column_name].unique()
    unique_values_amount = len(unique_values)
    return (unique_values, unique_values_amount)
    
def log_unique_values_and_their_amount_for_column(column_name, should_be_amount_shown = True):
    unique_values, unique_values_amount = get_unique_values_and_their_amount_for_column(column_name)
    print("Уникальные значения типа {}: {}".format(column_name, unique_values))
    if should_be_amount_shown:
        print("Колличeство уникальных значений типа {}: {}".format(column_name, unique_values_amount))

def compare_unique_type_to_id_amount(type_column_name, id_column_name):
    type_unique_values_amount = get_unique_values_and_their_amount_for_column(type_column_name)[1]
    id_unique_values_amount = get_unique_values_and_their_amount_for_column(id_column_name)[1]
    compared = type_unique_values_amount == id_unique_values_amount
    if compared: print("Колличество уникальных типов {} равно колличеству уникальных id {}".format(type_column_name, id_column_name))
    else: print("Колличество уникальных типов {} не равно колличеству уникальных id {}".format(type_column_name, id_column_name))
        
# Сравним колличнство типов и их id для образования, для поиска возможных повторений
log_unique_values_and_their_amount_for_column('education')
log_unique_values_and_their_amount_for_column('education_id')
compare_unique_type_to_id_amount('education', 'education_id')

print("----------------------")

# Сравним колличнство типов и их id для семейного положения, для поиска возможных повторений
log_unique_values_and_their_amount_for_column('family_status')
log_unique_values_and_their_amount_for_column('family_status_id')
compare_unique_type_to_id_amount('family_status', 'family_status_id')

print("----------------------")

# Посмотрим уникальные значения пола
log_unique_values_and_their_amount_for_column('gender', False)

# Посмотрим уникальные значения типа дохода
log_unique_values_and_their_amount_for_column('income_type', False)

# Посмотрим уникальные значения столбца дети
log_unique_values_and_their_amount_for_column('children', False)

# Посмотрим уникальные значения столбцп долг
log_unique_values_and_their_amount_for_column('debt', False)

print("----------------------")

# Проверим минимальный и максимальный возраст
print("Минимальный возраст {}".format(solvency_data['full_years'].min()))
print("Максимальный возраст {}".format(solvency_data['full_years'].max()))

Уникальные значения типа education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Колличeство уникальных значений типа education: 15
Уникальные значения типа education_id: [0 1 2 3 4]
Колличeство уникальных значений типа education_id: 5
Колличество уникальных типов education не равно колличеству уникальных id education_id
----------------------
Уникальные значения типа family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Колличeство уникальных значений типа family_status: 5
Уникальные значения типа family_status_id: [0 1 2 3 4]
Колличeство уникальных значений типа family_status_id: 5
Колличество уникальных типов family_status равно колличеству уникальных id family_status_id
----------------------
Уникальные значения типа gender: ['F' 'M' 'XNA']
Уникальные значения т

*Перед поиском и удалением дубликатов разберемся с аномалиями в стообцах children и full_years, так как детей не может быть -1,
а значение 20 довольно велико. Что касательно возраста, то кредиты вероятнее всего не даются детям, поэтому от 0 до 18 значения тоже
из ряда каких-то аномалий.*

In [9]:
# Проверим значения по возрасту 
# Закомментировано для экономии места
# print(solvency_data['full_years'].value_counts())

# Как видим, присутствую 101 человек без указанного возраста, значит пропуски скорее всего были заполнены нулями
# Исходя из малого количнства таких людей этими данными в принципе можно принебречь
zero_age_people_to_all = len(solvency_data[solvency_data['full_years'] == 0]) / len(solvency_data)
print("Колличество людей с нулевым возрастом в отношении всего количнства: {:.2%}".format(zero_age_people_to_all))

solvency_data = solvency_data[solvency_data['full_years'] != 0]
print("Колличество людей с нулевым возрастом после дропа: {}".format(len(solvency_data[solvency_data['full_years'] == 0])))

Колличество людей с нулевым возрастом в отношении всего количнства: 0.47%
Колличество людей с нулевым возрастом после дропа: 0


In [10]:
# Проверим значения по колличеству детей 
print(solvency_data['children'].value_counts() / len(solvency_data) * 100)

# Аномальные значения в -1 и 20 занимают малую часть выборки, поэтому тоже можно ими принеьречь
solvency_data = solvency_data[(solvency_data['children'] != -1) & (solvency_data['children'] != 20)]
abnormal_children_amount_mask = len(solvency_data[(solvency_data['children'] == -1) | (solvency_data['children'] == 20)])
print("Колличество аномальных значение по детям после удаления: {}".format(abnormal_children_amount_mask))

 0     65.720687
 1     22.414115
 2      9.531367
 3      1.530993
 20     0.350075
-1      0.219380
 4      0.191374
 5      0.042009
Name: children, dtype: float64
Колличество аномальных значение по детям после удаления: 0


*Теперь можем заняться регистром типов образования.*

In [11]:
# Приведем образлвание к единому регистру
solvency_data['education'] = solvency_data['education'].str.lower()

# Сравним колличнство типов и их id для образования
log_unique_values_and_their_amount_for_column('education')
log_unique_values_and_their_amount_for_column('education_id')
compare_unique_type_to_id_amount('education', 'education_id')

print("----------------------")

# Поищем дупликаты
duplciates_amount = solvency_data.duplicated().sum()
print("Колличество дубликатов в процентах {:.2%}".format(duplciates_amount / len(solvency_data)))

print("----------------------")

# Вопрос в том, а дубоикаты ли это или просто так совпало, что у людей одинаковые данные, 
# но все же их колличество мало, всего 0.33%, то можно их дропнуть

solvency_data = solvency_data.drop_duplicates().reset_index(drop=True)
print("Колличество дубликатов после дропа: {}".format(solvency_data.duplicated().sum()))

Уникальные значения типа education: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
Колличeство уникальных значений типа education: 5
Уникальные значения типа education_id: [0 1 2 3 4]
Колличeство уникальных значений типа education_id: 5
Колличество уникальных типов education равно колличеству уникальных id education_id
----------------------
Колличество дубликатов в процентах 0.33%
----------------------
Колличество дубликатов после дропа: 0


**Вывод** 

Перед обработкой дубликатов был проведен анализ уникальности полей и возможных ранее не замеченных аномалий. 

В процессе поиска обнаружено написание одного и того же типа образования в разных регистрах, что было преведено к единой форме. Также замечены люди без указания возраста, а точнее с возрастом 0, люди с колличеством детей -1 и 20, первый случай не возможен, второй довольно экстремальный. 

Все аномальные значения в связи их малого влияния на общую картину(лишь десятые и сотые от общего колличества) были удалены. Дубликаты в колличнстве 0.33% были также удалены из таблицы. 

### Лемматизация

In [ ]:
stemmer = Mystem() 

# Проверим уникальные щначения целей кредита
log_unique_values_and_their_amount_for_column('purpose', False)

# Посмотрим использованные лемы среди уникальных целей кредита  
purpose_lemmas = []
for purpose in get_unique_values_and_their_amount_for_column('purpose')[0]:
    purpose_lemmas += stemmer.lemmatize(purpose)
    
print("----------------------")
print(Counter(purpose_lemmas))
print("----------------------")
print(Counter(stemmer.lemmatize(" ".join(solvency_data['purpose'].tolist()))))

Уникальные значения типа purpose: ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образовани

*Среди лем можно выделить основные цели кредита: **жилье, недвижимость, автомобиль, свадьба, образование**. Желье и недвижимость отнесем к недвижимости*

In [ ]:
# Проверим гипотезу по разделению лем: недвижимость [жилье, недвижимость], автомобиль, свадьба, образование

def check_purpose_in_text(text, purpose):
    lemmatized = stemmer.lemmatize(text)
    
    if type(purpose) is str:
        return purpose in lemmatized
    
    if type(purpose) is list:
        for p in purpose:
            if p in lemmatized:
                return True
            
    return False

def check_purpose_in_dataset(purpose):
    return solvency_data['purpose'].apply(check_purpose_in_text, purpose = purpose)

def is_lemmatization_right():
    purposes_amount = check_purpose_in_dataset(['жилье', 'недвижимость', 'автомобиль', 'свадьба', 'образование']).sum()
    purposes_amount_are_equal_to_all_data = purposes_amount == len(solvency_data)
    
    if purposes_amount_are_equal_to_all_data: print('Лемматизация проведена верно')
    else: print('Лемматизация проведена не верно')
        
is_lemmatization_right()

**Вывод**

Проведя лемматизацию по целям было выясненно, что основными целями взятия кредита являются: недвижимость (жилье, недвижимость), 'автомобиль', 'свадьба', 'образование'. Сравнив колличество данных по целяи с общим колличеством было получено равенство, что говорит о полном разделении данных по целям.  


### Категоризация данных

In [ ]:
# Разобьем наши данные для анализа по категориям зарплаты, возраста и цели кредита

def get_monthly_income_category(income):
    if income < 32000:
        return 'бедный'
    elif income < 100000: 
        return 'достаточный'
    elif income < 300000:
        return 'средний'
    elif income < 500000:
        return 'зажиточный'
    else: return 'богатый'
    
def get_purpose_category(purpose_text):
    if check_purpose_in_text(purpose_text, ['жилье', 'недвижимость']):
        return 'недвижимость'
    elif check_purpose_in_text(purpose_text, 'автомобиль'):
        return 'автомобиль'
    elif check_purpose_in_text(purpose_text, 'свадьба'):
        return 'свадьба'
    elif check_purpose_in_text(purpose_text, 'образование'):
        return 'образование'
    else: return 'other'
    
def get_age_category(age):
    if age < 30:
        return 'молодой'
    elif age < 65:
        return 'зрелый'
    else:
        return 'старик'
    
solvency_data['monthly_income_category'] = solvency_data['monthly_income'].apply(get_monthly_income_category)
solvency_data['purpose_category'] = solvency_data['purpose'].apply(get_purpose_category)
solvency_data['age_category'] = solvency_data['full_years'].apply(get_age_category)
solvency_data.loc[:50, :]
    

**Вывод**

Была произведено приблизительное разбиение по категориям зарплаты (от бедных до богатых), возраста (от молодежи до стариков) и цели кредита (недвижимость, автомобиль, свадьба, образование). Данные категории нам понадобятся для анализа зависимостей выплат кредитов. 

In [ ]:
### Код ревьювера ###

# 50%-ый квантиль (перцентиль) == медиане, 25% и 75% делят оставшиеся половины еще раз поровну
solvency_data['monthly_income'].quantile([0.25, 0.5, 0.75])

## Шаг 3. Ответьте на вопросы

In [ ]:
# Определим функции для совершения выводов по должникам

def get_category_to_all_percentage_for_column(column_name):
    return solvency_data[column_name].value_counts() / len(solvency_data) * 100

def get_category_debt_to_all_in_category_percentage_for_column(column_name):
    return solvency_data.groupby(column_name)['debt'].sum() / solvency_data.groupby(column_name)['debt'].count() * 100

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
print("Отношение по категориям колличества детей ко всей выборке")
print(get_category_to_all_percentage_for_column('children'))

print("----------------------")

print("Отношение должников ко всем людям по категории колличества детей")
print(get_category_debt_to_all_in_category_percentage_for_column('children')) 

**Вывод**

Основная масса людей берущих кредиты это те у кого нет детей (66%) и люди с одним ребенком (22%), и как видим, что чем больше детей, тем меньше люди берут кредиты, скорее всего из-за того, что отдавать их будет сложно, так как дети это огромные вложения. 

При этом задолженность по кредиту в категориях находится на одном уровне (около 9%), с некоторыми улучшением у людей без детей (7.5%), так как отдавать им скорее всего легче. При этом факт, что среди тех у кого 5 детей нет задолженности скорее всего может быть связан с тем, что люди вообще не решаются на взятие кредита (лишь 0.04%), да и людей таких мало и если они уже кредит взяли, то скорее всего на это есть ресурсы.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
print("Отношение по категориям семейного статуса ко всей выборке")
print(get_category_to_all_percentage_for_column('family_status'))

print("----------------------")

print("Отношение должников ко всем людям по категории семейного статуса")
print(get_category_debt_to_all_in_category_percentage_for_column('family_status')) 

print("----------------------")

not_married_or_civil_marriage_mask = ((solvency_data['family_status'] == 'Не женат / не замужем') | (solvency_data['family_status'] == 'гражданский брак'))
not_married_or_civil_marriage_with_debt_mask = not_married_or_civil_marriage_mask & (solvency_data['debt'] == 1) 

print('Распределение по возрасту, тех кто "Не женат / не замужем" или в "гражданском браке", среди тех кто задолжал кредиты к взявшим')
solvency_data[not_married_or_civil_marriage_with_debt_mask]['age_category'].value_counts() / solvency_data[not_married_or_civil_marriage_mask]['age_category'].value_counts()

**Вывод**

Кредиты берут в своей основной массе те, кто живет вместе, будь то гражданский брак или семейная пара (19.37% и 57.52% соответственно), так как совместная жизнь нуждается в разных инвестициях.

Касательно возвратов кредитов, то их задалживают в большей мере не женатые либо те, кто в гражданском браке (оба около 9%). Причиной того может быть то, что в большинстве своем это молодые еще люди, которые могут не имеют стабильного заработка, а также зрелые, которые переоценивают возможности.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
print("Отношение по категориям зароботка ко всей выборке")
print(get_category_to_all_percentage_for_column('monthly_income_category'))

print("----------------------")

print("Отношение должников ко всем людям по категории зароботка")
print(get_category_debt_to_all_in_category_percentage_for_column('monthly_income_category')) 

**Вывод**

Первое, что прослеживается это то, что бедные и богатые довольно мало берут кредиты (0.16% и 1.04% соответственно), скорее всего первым не дают, а вторые в них не особо нуждаются. Больше всего кредитов беруи люди со средним достатком (72.25%). 

Взяли отношение должников к общему колличеству в категории и увидели, что люди с достаточным и средним заработком задалживают больше кредитов (8%), так как они их в принципе берут больше, но и в то же время возможно, что у них нет большого запаса, то ли они переоценивают свои возможности.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
print("Отношение по категориям цели кредита ко всей выборке")
print(get_category_to_all_percentage_for_column('purpose_category'))

print("----------------------")

print("Отношение должников ко всем людям по категории цели кредита")
print(get_category_debt_to_all_in_category_percentage_for_column('purpose_category')) 

**Вывод**

Больше всего люди берут кредиты на нежвижимость, так как крышу над головой, либо стабильную инвестицию хотят иметь все, и таких у нас здесь 50%, после чего идет автомобиль с 20%, образование с 18%(толи люди не привыкли в него еще вкладывать, толи потому что от части оно много где бесплатное), ну и на последнем месте свадьба с 10%.

Недвижимость и свадьба - цели по которым задалживают меньше, так как на свадьбе скорее всего деньги возвращаются в виде подарков, а что касательно недвижимости, то ее можно либо сдавать, либо люди в принципе не хотят потерять свое кровно заработанное жилье (в обоих случаях около 9%). С образованием и автомобилем дела по хуже, так как автомобиль это лишь затраты, а образование возвращает затраты со временем.

## Шаг 4. Общий вывод

В данном проекте была проведена предобработка данных, в ходе которой мы избавились от отрицательных значений и слишком большого количества дней в стаже "days_employed" методом замены на медиану, также эту методику мы применили для обработки пропусков в месячной зарплате "monthly_income", оба столбца были преобразованы в целочисленный тип. Значения возраста равные 0, так же как и отрицательное количество детей, либо же колличество в размере 20 были отброшены в связи своей не значительности (0.4%, 0.3% и 0.2% соответственно) для всей выборки. Привели категории образования к единому формату (в нижнем регистре), также удалили дубликаты (0.33%). 

С помощью лемматизации мы вычислили, что основными целями взятия кредита являются недвижимость, автомобиль, образование и свадьба, добавив данную колонку, также мы категоризировали людей по зароботку и возрасту.

Проверив кто задалживает, а кто нет сложили следующий портрет:

- Идеальным человеком, для взятия кредита является тот у кого нет детей либо их пять, он богат или зажиточен, в разводе либо вдовец с целью кредита на свадьбу либо недвижимость.
- Антогонистом идеалу является человек с одним до четырех детей, среднего заработка, который не женат либо в гражданском браке и берет кредит на автомобиль.